# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/sarakleine-kracht/Desktop/python-api-challenge/SK_code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,t'ongch'on-up,38.9539,127.8917,-7.50,61,7,3.39,KP,1704660955
1,1,sodo,6.8642,37.7628,16.66,59,46,1.92,ET,1704660955
2,2,port mathurin,-19.6833,63.4167,27.38,75,45,3.82,MU,1704660918
3,3,port-aux-francais,-49.3500,70.2167,7.27,95,98,14.25,TF,1704660956
4,4,ariquemes,-9.9133,-63.0408,27.24,72,57,2.26,BR,1704660957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lat", 
    "Lng", 
    geo = True, 
    titles = "OSM", 
    frame_width = 500, 
    frame_height = 500,
    size = "Humidity", 
    scale = 0.5, 
    color = "City"
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Wind Speed'] < 2)]
# Drop any rows with null values
ideal_weather_df.dropna()
# Display sample data
# YOUR CODE HERE
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,sodo,6.8642,37.7628,16.66,59,46,1.92,ET,1704660955
5,5,port moresby,-9.4431,147.1797,23.68,94,20,1.54,PG,1704660957
10,10,la'ie,21.6477,-157.9253,27.81,74,42,1.79,US,1704660958
20,20,bengkulu,-3.8004,102.2655,24.48,88,100,1.40,ID,1704660962
43,43,papatowai,-46.5619,169.4708,15.83,67,100,1.15,NZ,1704660971
...,...,...,...,...,...,...,...,...,...,...
552,552,pariaman,-0.6190,100.1200,25.03,87,99,1.53,ID,1704661160
555,555,ballina,-28.8667,153.5667,23.21,80,94,0.66,AU,1704661161
569,569,kande,9.9578,1.0447,27.22,17,1,1.63,TG,1704661166
573,573,abong mbang,3.9833,13.1833,20.96,28,9,0.88,CM,1704661168


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df.drop(columns= ['Max Temp','Cloudiness','Wind Speed','Date'])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df.insert(6, "Hotel Name", " ")
# Display sample data
hotel_df.head(15)

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
1,1,sodo,6.8642,37.7628,59,ET,
5,5,port moresby,-9.4431,147.1797,94,PG,
10,10,la'ie,21.6477,-157.9253,74,US,
20,20,bengkulu,-3.8004,102.2655,88,ID,
43,43,papatowai,-46.5619,169.4708,67,NZ,
51,51,vanimo,-2.6741,141.3028,87,PG,
57,57,bredasdorp,-34.5322,20.0403,69,ZA,
60,60,wanning,18.8003,110.3967,80,CN,
73,73,masuguru,-11.3667,38.4167,97,TZ,
76,76,labuan,5.2767,115.2417,100,MY,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {
    "categories":category,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = hotel_df.loc[index, ('Lng')]
    latitude = hotel_df.loc[index, ('Lat')]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sodo - nearest hotel: No hotel found
port moresby - nearest hotel: The Sanctuary Hotel and Spa
la'ie - nearest hotel: No hotel found
bengkulu - nearest hotel: Dena Hotel
papatowai - nearest hotel: No hotel found
vanimo - nearest hotel: CBC Guesthouse
bredasdorp - nearest hotel: Victoria Hotel
wanning - nearest hotel: No hotel found
masuguru - nearest hotel: No hotel found
labuan - nearest hotel: Labuan Avenue Hotel
kone - nearest hotel: Pacifik Appartels
palopo - nearest hotel: Hotel Mulia Indah
kabalo - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
nova olinda do norte - nearest hotel: No hotel found
kokoda - nearest hotel: No hotel found
gambela - nearest hotel: Grand Hotel
georgetown - nearest hotel: Page 63 hostel
malinyi - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: Hotel Terraço
myeik - nearest hotel: Green Eyes Hotel
castillos - nearest hotel: A mi gente
aripuana - nearest hotel: No hotel found
lumajang - neare

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
1,1,sodo,6.8642,37.7628,59,ET,No hotel found
5,5,port moresby,-9.4431,147.1797,94,PG,The Sanctuary Hotel and Spa
10,10,la'ie,21.6477,-157.9253,74,US,No hotel found
20,20,bengkulu,-3.8004,102.2655,88,ID,Dena Hotel
43,43,papatowai,-46.5619,169.4708,67,NZ,No hotel found
...,...,...,...,...,...,...,...
552,552,pariaman,-0.6190,100.1200,87,ID,HOTEL NAN TONGGA
555,555,ballina,-28.8667,153.5667,80,AU,Ballina Beach Resort
569,569,kande,9.9578,1.0447,17,TG,No hotel found
573,573,abong mbang,3.9833,13.1833,28,CM,Bagofit Sun City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lat", 
    "Lng",
    geo = True, 
    titles = "OSM", 
    frame_width = 500, 
    frame_height = 500,
    size = "Humidity", 
    scale = 0.5, 
    color = "City",
    hover_cols = "Hotel Name")
# Display the map
# YOUR CODE HERE

map_plot_2

:Points   [Lat,Lng]   (City,Humidity,Hotel Name)